# 양돈 모돈 데이터 파일 전처리 및 시각화

In [4]:
import pandas as pd

# 데이터 불러오기
file_path = "../../data/2023년 양돈 모돈 데이터.csv"
df = pd.read_csv(file_path)

In [5]:
df

In [6]:
# 날짜 변환
df['분만일'] = pd.to_datetime(df['분만일'], format='%Y%m%d', errors='coerce')

# 연도, 월, 계절 컬럼 추가
df['연도'] = df['분만일'].dt.year
df['월'] = df['분만일'].dt.month

In [7]:
df

In [8]:
# 계절 매핑
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

df['계절'] = df['월'].apply(get_season)

In [9]:
df

In [10]:
# 분석 대상 컬럼 중 일부 NaN 제거
df_filtered = df.dropna(subset=['실산자수', '이유두수'])

# 산차별 실산자수/이유두수 평균
by_parity = df_filtered.groupby('산차')[['실산자수', '이유두수']].mean().reset_index()

# 월별 실산자수/이유두수 평균
by_month = df_filtered.groupby('월')[['실산자수', '이유두수']].mean().reset_index()

# 계절별 실산자수/이유두수 평균
by_season = df_filtered.groupby('계절')[['실산자수', '이유두수']].mean().reset_index()
# 계절 순서 재정렬: 봄 → 여름 → 가을 → 겨울
season_order = ['겨울', '봄', '여름', '가을']
by_season_sorted = by_season.set_index('계절').loc[season_order].reset_index()

In [11]:
by_parity

In [12]:
import matplotlib.pyplot as plt
import koreanize_matplotlib

In [13]:
def plot_grouped_data(df, x, y_list, title):
    for y in y_list:
        plt.plot(df[x], df[y], marker='.', label=y)
    plt.title(title)
    plt.xlabel(x)
    plt.ylabel('두수')
    plt.legend()
    plt.grid(True)
    plt.show()

In [14]:
# 시각화 출력
plot_grouped_data(by_parity, '산차', ['실산자수', '이유두수'], '산차에 따른 번식 성적 (실산자수 / 이유두수)')

In [15]:
plot_grouped_data(by_month, '월', ['실산자수', '이유두수'], '월별 번식 성적 (실산자수 / 이유두수)')

In [16]:
plot_grouped_data(by_season_sorted, '계절', ['실산자수', '이유두수'], '계절별 번식 성적 (실산자수 / 이유두수)')